# Requirements

In [1]:
!pip install wandb transformers[torch] torch pandas datasets evaluate sacrebleu scikit-learn
!pip install accelerate -U

In [2]:
import logging
import math
from functools import partial
from itertools import chain

import datasets
import evaluate
import wandb
from transformers import (
    XLMRobertaTokenizer,
    XLMRobertaForMaskedLM,
    XLMRobertaXLForMaskedLM,
    DataCollatorForLanguageModeling,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    PreTrainedTokenizerFast
)


# WandB

In [3]:
wandb.login(key="d50c52de8f8f7a0f7afebb827fbfbdf1e506cb2b")

wandb: Currently logged in as: simmons (coles). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Constants

In [4]:
#BASE_PATH = "/workspace/"
#MODEL_ID = "./results/encoder/best_model"

# Dataset

In [5]:
raw_dataset = datasets.load_dataset("colesimmons/SumTablets")
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'period', 'genre', 'transliteration', 'glyph_names', 'glyphs'],
        num_rows: 82452
    })
    validation: Dataset({
        features: ['id', 'period', 'genre', 'transliteration', 'glyph_names', 'glyphs'],
        num_rows: 4577
    })
    test: Dataset({
        features: ['id', 'period', 'genre', 'transliteration', 'glyph_names', 'glyphs'],
        num_rows: 4577
    })
})

In [6]:
raw_dataset["train"][0]

{'id': 'P119622',
 'period': 'Ur III',
 'genre': 'Administrative',
 'transliteration': '<SURFACE>\n2(u) udu\nki nin-še₃\nki uš-mu-ta\nkišib₃ ensi₂-ka\n<SURFACE>\nša₃ uri₅{ki}ma\niti ezem-mah\nmu en eridu{ki} ba-hun\n<SURFACE>\n<COLUMN>\n{d}amar{d}suen\nlugal kal-ga\nlugal uri₅{ki}ma\nlugal an ub-da limmu₂-ba\n<COLUMN>\na-kal-la\nensi₂\numma{ki}\n<unk> zu',
 'glyph_names': '<SURFACE> \n |U.U| LU \n KI |SAL.TUG₂| EŠ₂ \n KI UŠ MU TA \n DUB |PA.TE.SI| KA \n <SURFACE> \n ŠA₃ |ŠEŠ.AB| KI MA \n |UD×(U.U.U)| EZEN MAH \n MU EN NUN KI BA EŠ₂ \n <SURFACE> \n <COLUMN> \n AN AMAR AN |EN.ZU| \n LUGAL KAL GA \n LUGAL |ŠEŠ.AB| KI MA \n LUGAL AN UB DA LIMMU₂ BA \n <COLUMN> \n A KAL LA \n |PA.TE.SI| \n |GIŠ.KUŠU₂| KI \n <unk> ZU',
 'glyphs': '<SURFACE>\n𒎙𒇻\n𒆠𒎏𒂠\n𒆠𒍑𒈬𒋫\n𒁾𒉺𒋼𒋛𒅗\n<SURFACE>\n𒊮𒋀𒀊𒆠𒈠\n𒌗𒂡𒈤\n𒈬𒂗𒉣𒆠𒁀𒂠\n<SURFACE>\n<COLUMN>\n𒀭𒀫𒀭𒂗𒍪\n𒈗𒆗𒂵\n𒈗𒋀𒀊𒆠𒈠\n𒈗𒀭𒌒𒁕𒇹𒁀\n<COLUMN>\n𒀀𒆗𒆷\n𒉺𒋼𒋛\n𒄑𒆵𒆠\n<unk>𒍪'}

In [7]:
def oversample_non_administrative(dataset_):
    admin_examples = dataset_.filter(lambda example: example['genre'] == 'Administrative')
    print("Num admin: ", len(admin_examples))
    non_admin_examples = dataset_.filter(lambda example: example['genre'] != 'Administrative')
    print("Num non-admin: ", len(non_admin_examples))
    
    oversampling_factor = 10
    
    oversampled_non_admin = datasets.concatenate_datasets([non_admin_examples] * oversampling_factor)
    print("Num non-admin after: ", len(oversampled_non_admin))
    
    balanced_dataset = datasets.concatenate_datasets([admin_examples, oversampled_non_admin])
    balanced_dataset = balanced_dataset.shuffle(seed=42)
    return balanced_dataset

oversampled = raw_dataset
oversampled["train"] = oversample_non_administrative(oversampled['train'])
oversampled

Num admin:  77193
Num non-admin:  5259
Num non-admin after:  52590


DatasetDict({
    train: Dataset({
        features: ['id', 'period', 'genre', 'transliteration', 'glyph_names', 'glyphs'],
        num_rows: 129783
    })
    validation: Dataset({
        features: ['id', 'period', 'genre', 'transliteration', 'glyph_names', 'glyphs'],
        num_rows: 4577
    })
    test: Dataset({
        features: ['id', 'period', 'genre', 'transliteration', 'glyph_names', 'glyphs'],
        num_rows: 4577
    })
})

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("colesimmons/SumerianGlyphTokenizer_Roberta", force_download=True)
tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.85k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='colesimmons/SumerianGlyphTokenizer_Roberta', vocab_size=632, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['<SURFACE>', '<COLUMN>', '<BLANK_SPACE>', '<RULING>', '\n', '...']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=False,

In [9]:
def tokenize_and_chunk(dataset, tokenizer):
    """Tokenize the tablets, remove unused columns,
    and split into chunks of length seq_len
    """
    
    seq_len = 64
    
    def tokenize(examples):
        tokenized = tokenizer(
            examples["glyphs"],
            padding=False,
            truncation=False,
        )
        tokenized["special_tokens_mask"] = [
            tokenizer.get_special_tokens_mask(input_ids, already_has_special_tokens=True)
            for input_ids in tokenized["input_ids"]
        ]
        return tokenized

    def split_into_chunks(examples):  
        chunks = {key: [] for key in examples.keys()}
        for k in examples.keys():
            for example in examples[k]:                
                if len(example) <= seq_len:
                    chunks[k].append(example)
                elif len(example) <= 2 * seq_len:
                    chunks[k].append(example[:seq_len])
                    chunks[k].append(example[-seq_len:])
                else:
                    for i in range(0, len(example) - seq_len + 1, seq_len - 10):
                        chunks[k].append(example[i:i + seq_len])
                    chunks[k].append(example[-seq_len:])

        return chunks

    columns_to_remove = dataset.column_names["train"]
    dataset = dataset.map(tokenize, batched=True, remove_columns=columns_to_remove)
    dataset = dataset.map(split_into_chunks, batched=True)
    return dataset

dataset = tokenize_and_chunk(oversampled, tokenizer)

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 334679
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 9826
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 10149
    })
})

In [11]:
_idx = 0
print(dataset["train"][_idx])
print(tokenizer.decode(dataset["train"][_idx]["input_ids"], skip_special_tokens=False))
#print()
#print(tokenizer.convert_ids_to_tokens(472))

{'input_ids': [0, 6, 5, 476, 560, 196, 482, 344, 113, 5, 178, 281, 267, 464, 12, 113, 5, 161, 203, 399, 225, 128, 5, 565, 196, 482, 560, 113, 5, 10, 474, 3, 5, 6, 5, 10, 58, 64, 5, 389, 330, 345, 334, 373, 5, 92, 275, 126, 366, 5, 345, 209, 209, 376, 401, 267, 10, 219, 496, 5, 6, 5, 275, 126], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'special_tokens_mask': [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0]}
<s><SURFACE>
𒌋𒐈𒄨𒌓𒈫𒂠
𒄑𒆵𒆠𒋼𒀀𒂠
𒃻𒄯𒊏𒅍𒂷
𒐍𒄨𒌓𒐈𒂠
...𒌈<unk>
<SURFACE>
...𒁍𒁕

In [12]:
# seeds 42, 20, 10, 5, 3, 50
# seeds 1 (emb + lm head), 2, 3,
# skip 8
shuffled = dataset.shuffle(seed=10)
shuffled

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 334679
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 9826
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 10149
    })
})

In [13]:
#id_to_token = {id: token for token, id in tokenizer.get_vocab().items()}
#print(id_to_token[250004])
#print(id_to_token[250003])

In [14]:
print([k for k in tokenizer.get_vocab().keys() if len(k) > 1])

['<unk>', '<pad>', '</s>', '<s>', '<BLANK_SPACE>', '<SURFACE>', '...', '<mask>', '<COLUMN>', '<RULING>']


In [15]:
#import os
#os.makedirs("/models")

# Model

In [16]:
# ------ METRICS -----
metric = evaluate.load("accuracy")

def preprocess_logits_for_metrics(logits, labels):
    return logits.argmax(dim=-1)

def compute_metrics(eval_preds: EvalPrediction):
    preds, labels = eval_preds
    labels = labels.reshape(-1)
    preds = preds.reshape(-1)
    mask = labels != -100
    labels = labels[mask]
    preds = preds[mask]
    return metric.compute(predictions=preds, references=labels)


# ------ MODEL -----
def init_model(
    *,
    model_id,
    attn_dropout_prob=0.1,
    hidden_dropout_prob=0.1,
    num_hidden_layers=12,
    num_attention_heads=12,
):
    
    model = XLMRobertaForMaskedLM.from_pretrained(
        model_id,
        attention_probs_dropout_prob=attn_dropout_prob,
        hidden_dropout_prob=hidden_dropout_prob,
        #num_hidden_layers=num_hidden_layers,
        #num_attention_heads=num_attention_heads,
    )
    print("Loading: ", model_id)
    
    # Freeze all layers in the model
    if model_id.startswith("/"):
        model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=128)
        for name, param in model.named_parameters():
            param.requires_grad = True
        for param in model.roberta.embeddings.parameters():
            param.requires_grad = True
        #for name, param in model.named_parameters():
            #if "layer." in name and int(name.split(".")[3]) >= 2:
                #param.requires_grad = True
        for param in model.lm_head.parameters():
            param.requires_grad = True
    else:
        # resize embeddings for new vocab
        print("Num parameters before: ", sum(p.numel() for p in model.parameters()))
        model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=128)
        print("Num parameters after: ", sum(p.numel() for p in model.parameters()))
        for param in model.parameters():
            param.requires_grad = False
        # Unfreeze the embedding layer
        for param in model.roberta.embeddings.parameters():
            param.requires_grad = True
        for param in model.lm_head.parameters():
            param.requires_grad = True
    
    for name, param in model.named_parameters():
        print(f"{name}: {param.requires_grad}")

    return model


def init_data_collator(*, mlm_prob):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm_probability=mlm_prob
    )


def init_training_args(
    *,
    eval_batch_size,
    eval_steps,
    lr,
    logging_steps,
    num_epochs,
    run_name,
    save_steps,
    save_total_limit,
    train_batch_size,
    warmup_steps,
):
    return TrainingArguments(
        # Run info
        output_dir=f"/models/{run_name}",
        run_name=run_name,
        # Logging
        logging_steps=logging_steps,
        # Saving
        save_steps=save_steps,
        save_total_limit=save_total_limit,
        # Train
        learning_rate=lr,
        num_train_epochs=num_epochs,
        per_device_train_batch_size=train_batch_size,
        warmup_steps=warmup_steps,
        # Eval
        eval_steps=eval_steps,
        per_device_eval_batch_size=eval_batch_size,
        evaluation_strategy="steps",
        # Return model
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
    )

In [17]:
def train(
    *,
    run_name,
    model_id,
    attn_dropout_prob,
    hidden_dropout_prob,
    lr,
    mlm_prob,
    # Meta
    num_epochs,
    train_batch_size=128,
    eval_batch_size=64,
    warmup_steps,
):
    model = init_model(
        model_id=model_id,
        attn_dropout_prob=attn_dropout_prob,
        hidden_dropout_prob=hidden_dropout_prob,
    )
    
    data_collator = init_data_collator(
        mlm_prob=mlm_prob,
    )

    training_args = init_training_args(
        eval_batch_size=eval_batch_size,
        eval_steps=100,
        lr=lr,
        logging_steps=10,
        num_epochs=num_epochs,
        run_name=run_name,
        save_steps=500,
        save_total_limit=2,
        train_batch_size=train_batch_size,
        warmup_steps=warmup_steps,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=shuffled["train"],
        eval_dataset=shuffled["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        preprocess_logits_for_metrics=preprocess_logits_for_metrics
    )

    wandb.init(
        project="glyph_encoder",
        #notes="test",
        config={"mlm_prob": mlm_prob}
    )
    
    train_result = trainer.train()
    trainer.save_model(f"/models/{run_name}/best_model")

    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()

    metrics = trainer.evaluate()
    
    #try:
    #    perplexity = math.exp(metrics["eval_loss"])
    #except OverflowError:
    #    perplexity = float("inf")

    #metrics["perplexity"] = perplexity
    #wandb.log(metrics)

    #trainer.log_metrics("eval", metrics)
    #trainer.save_metrics("eval", metrics)

    wandb.finish()
    return

In [ ]:
lrs = [1e-4]
attn_dropout_probs = [0.1]
hidden_dropout_probs = [0.1]
mlm_probs = [0.05]
num_epochs = 5

#model_id = "FacebookAI/xlm-roberta-base"
model_id = "/models/base_2_all_layers/best_model"

run_name = "base_2_all_layers_2"

warmup_steps = 300

for lr in lrs:
    for attn_dropout_prob in attn_dropout_probs:
        for hidden_dropout_prob in hidden_dropout_probs:
            for mlm_prob in mlm_probs:
                model = train(
                    #run_name=f"lr={lr}-attn_do={attn_dropout_prob}-hidden_do={hidden_dropout_prob}-mlm_prob={mlm_prob}",
                    model_id=model_id,
                    run_name=run_name,
                    attn_dropout_prob=attn_dropout_prob,
                    hidden_dropout_prob=hidden_dropout_prob,
                    lr=lr,
                    mlm_prob=mlm_prob,
                    num_epochs=num_epochs,
                    warmup_steps=warmup_steps,
                    train_batch_size=1024,
                )

Loading:  /models/base_2_all_layers/best_model
roberta.embeddings.word_embeddings.weight: True
roberta.embeddings.position_embeddings.weight: True
roberta.embeddings.token_type_embeddings.weight: True
roberta.embeddings.LayerNorm.weight: True
roberta.embeddings.LayerNorm.bias: True
roberta.encoder.layer.0.attention.self.query.weight: True
roberta.encoder.layer.0.attention.self.query.bias: True
roberta.encoder.layer.0.attention.self.key.weight: True
roberta.encoder.layer.0.attention.self.key.bias: True
roberta.encoder.layer.0.attention.self.value.weight: True
roberta.encoder.layer.0.attention.self.value.bias: True
roberta.encoder.layer.0.attention.output.dense.weight: True
roberta.encoder.layer.0.attention.output.dense.bias: True
roberta.encoder.layer.0.attention.output.LayerNorm.weight: True
roberta.encoder.layer.0.attention.output.LayerNorm.bias: True
roberta.encoder.layer.0.intermediate.dense.weight: True
roberta.encoder.layer.0.intermediate.dense.bias: True
roberta.encoder.layer.0.o

Step,Training Loss,Validation Loss,Accuracy
100,2.148100,1.560775,0.654318
200,2.121600,1.557898,0.656331
300,2.104900,1.584578,0.653005
400,2.014900,1.512970,0.662776
500,1.952400,1.484140,0.668525
600,1.831000,1.412836,0.678291
700,1.792000,1.363372,0.691120
800,1.727200,1.331165,0.695143
900,1.684700,1.343618,0.693680
1000,1.675300,1.291191,0.704911


In [75]:
model.push_to_hub("xlm-r-sumerian-glyphs")

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/colesimmons/xlm-r-sumerian-glyphs/commit/5a673dba8ea7af28fe93335e7db78ea5c4239dcc', commit_message='Upload XLMRobertaForMaskedLM', commit_description='', oid='5a673dba8ea7af28fe93335e7db78ea5c4239dcc', pr_url=None, pr_revision=None, pr_num=None)

# Misc

In [ ]:
 model = AutoModelForMaskedLM.from_pretrained(
    MODEL_ID,
)

for name, param in model.named_parameters():
    print(f"Layer: {name}, Frozen: {not param.requires_grad}")

In [71]:
model = XLMRobertaForMaskedLM.from_pretrained("./results/last_11_layers/best_model")
model

XLMRobertaForMaskedLM(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(640, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
     

In [74]:
from huggingface_hub import notebook_login
notebook_login()

In [194]:
import shutil

# Specify the path of the directory to delete
directory_path = 'wandb'

# Remove the directory and all its contents
shutil.rmtree(directory_path)

In [32]:
model = XLMRobertaForMaskedLM.from_pretrained(
        model_id,
        attention_probs_dropout_prob=attn_dropout_prob,
        hidden_dropout_prob=hidden_dropout_prob,
        #num_hidden_layers=num_hidden_layers,
        #num_attention_heads=num_attention_heads,
    )
print("Loading: ", model_id)

for name, param in model.named_parameters():
    param.requires_grad = False
for param in model.roberta.embeddings.parameters():
    param.requires_grad = True
for name, param in model.named_parameters():
    if "layer." in name and int(name.split(".")[3]) >= 8:
        param.requires_grad = True
for param in model.lm_head.parameters():
    param.requires_grad = True
# Freeze all layers in the model
for name, param in model.named_parameters():
    print(f"{name}: {param.requires_grad}")


Loading:  ./results/new_tokenizer/best_model
roberta.embeddings.word_embeddings.weight: True
roberta.embeddings.position_embeddings.weight: True
roberta.embeddings.token_type_embeddings.weight: True
roberta.embeddings.LayerNorm.weight: True
roberta.embeddings.LayerNorm.bias: True
roberta.encoder.layer.0.attention.self.query.weight: False
roberta.encoder.layer.0.attention.self.query.bias: False
roberta.encoder.layer.0.attention.self.key.weight: False
roberta.encoder.layer.0.attention.self.key.bias: False
roberta.encoder.layer.0.attention.self.value.weight: False
roberta.encoder.layer.0.attention.self.value.bias: False
roberta.encoder.layer.0.attention.output.dense.weight: False
roberta.encoder.layer.0.attention.output.dense.bias: False
roberta.encoder.layer.0.attention.output.LayerNorm.weight: False
roberta.encoder.layer.0.attention.output.LayerNorm.bias: False
roberta.encoder.layer.0.intermediate.dense.weight: False
roberta.encoder.layer.0.intermediate.dense.bias: False
roberta.encoder

# Train Tokenizer

In [6]:
from tokenizers import Tokenizer, decoders, models, normalizers, pre_tokenizers, trainers, processors
from transformers import AutoTokenizer
from tokenizers.models import BPE

In [7]:
ALL_SPECIAL_TOKS = [
    '<s>',
    '<pad>',
    '</s>',
    '<unk>',
    '<mask>',
    "\n",
    "<SURFACE>",
    "<COLUMN>",
    "<BLANK_SPACE>",
    "<RULING>",
    "..."
]                        

def get_glyph_texts():
    glyph_texts = []
    for split in dataset.keys():
        for example in dataset[split]:
            text = example["glyphs"]
            for token_ in ALL_SPECIAL_TOKS:
                text = text.replace(token_, "")
            glyph_texts.append(text)
    print(glyph_texts[0])
    return glyph_texts

print(dataset["train"][0])
glyph_texts = get_glyph_texts()

{'id': 'P119622', 'period': 'Ur III', 'genre': 'Administrative', 'transliteration': '<SURFACE>\n2(u) udu\nki nin-še₃\nki uš-mu-ta\nkišib₃ ensi₂-ka\n<SURFACE>\nša₃ uri₅{ki}ma\niti ezem-mah\nmu en eridu{ki} ba-hun\n<SURFACE>\n<COLUMN>\n{d}amar{d}suen\nlugal kal-ga\nlugal uri₅{ki}ma\nlugal an ub-da limmu₂-ba\n<COLUMN>\na-kal-la\nensi₂\numma{ki}\n<unk> zu', 'glyph_names': '<SURFACE> \n |U.U| LU \n KI |SAL.TUG₂| EŠ₂ \n KI UŠ MU TA \n DUB |PA.TE.SI| KA \n <SURFACE> \n ŠA₃ |ŠEŠ.AB| KI MA \n |UD×(U.U.U)| EZEN MAH \n MU EN NUN KI BA EŠ₂ \n <SURFACE> \n <COLUMN> \n AN AMAR AN |EN.ZU| \n LUGAL KAL GA \n LUGAL |ŠEŠ.AB| KI MA \n LUGAL AN UB DA LIMMU₂ BA \n <COLUMN> \n A KAL LA \n |PA.TE.SI| \n |GIŠ.KUŠU₂| KI \n <unk> ZU', 'glyphs': '<SURFACE>\n𒎙𒇻\n𒆠𒎏𒂠\n𒆠𒍑𒈬𒋫\n𒁾𒉺𒋼𒋛𒅗\n<SURFACE>\n𒊮𒋀𒀊𒆠𒈠\n𒌗𒂡𒈤\n𒈬𒂗𒉣𒆠𒁀𒂠\n<SURFACE>\n<COLUMN>\n𒀭𒀫𒀭𒂗𒍪\n𒈗𒆗𒂵\n𒈗𒋀𒀊𒆠𒈠\n𒈗𒀭𒌒𒁕𒇹𒁀\n<COLUMN>\n𒀀𒆗𒆷\n𒉺𒋼𒋛\n𒄑𒆵𒆠\n<unk>𒍪'}
𒎙𒇻𒆠𒎏𒂠𒆠𒍑𒈬𒋫𒁾𒉺𒋼𒋛𒅗𒊮𒋀𒀊𒆠𒈠𒌗𒂡𒈤𒈬𒂗𒉣𒆠𒁀𒂠𒀭𒀫𒀭𒂗𒍪𒈗𒆗𒂵𒈗𒋀𒀊𒆠𒈠𒈗𒀭𒌒𒁕𒇹𒁀𒀀𒆗𒆷𒉺𒋼𒋛𒄑𒆵𒆠𒍪


In [8]:
def get_unique_glyphs(dataset):
    unique_glyphs = set()

    for split in dataset.keys():
        for example in dataset[split]:
            text = example["glyphs"]
            for token_ in ALL_SPECIAL_TOKS:
                text = text.replace(token_, "")
            unique_glyphs |= set(text)
            
    print("Num unique glyphs: ", len(unique_glyphs))
    print(unique_glyphs)
    return unique_glyphs

unique_glyphs = get_unique_glyphs(dataset)
unique_glyph_count = len(unique_glyphs)

Num unique glyphs:  621
{'𒆝', '𒐒', '𒀶', '𒋤', '𒀉', '𒌜', '𒂉', '𒄤', '𒋡', '𒊯', '𒉪', '𒓻', '𒂶', '𒍺', '𒂁', '𒈨', '𒈤', '𒌺', '𒂢', '𒇑', '𒅘', '𒀪', '𒐕', '𒄆', '𒅥', '𒁡', '𒄱', '𒎗', '𒅝', '\xa0', '𒎐', '𒁰', '𒇦', '𒆑', '𒌈', '𒌆', '𒃎', '𒃴', '𒊗', '𒄪', '𒁯', '𒐀', '𒈬', '𒌢', '𒂃', '𒐋', '𒌍', '𒉢', '𒐝', '𒁕', '𒇻', '𒆈', '𒅌', '𒇽', '𒍨', '𒂀', '𒋀', '𒋸', '𒄖', '𒉈', '𒂕', '𒉽', '𒊨', '𒍋', '𒋄', '𒁻', '𒌋', '𒋃', '𒑙', '𒂷', '𒐙', '𒆚', '𒋞', '𒒬', '𒁛', '𒈽', '𒈣', '𒇲', '𒉘', '𒇿', '𒇡', '𒃘', '𒇁', '𒃶', '𒍦', '𒊲', '𒊭', '𒆶', '𒃞', '𒍇', '𒆀', '𒐚', '𒄽', '𒁼', '𒆢', '𒈸', '𒊫', '𒃱', '𒉀', '𒃣', '𒃕', '𒋙', '𒁎', '𒓼', '𒀲', '𒁈', '𒄮', '𒐂', '𒅍', '𒐜', '𒀇', '𒇚', '𒍖', '𒂣', '𒐰', '𒀸', '𒍩', '𒐈', '𒐮', '𒃰', '𒀋', '𒁦', '𒎏', '𒃟', '𒆤', '𒐗', '𒃠', '𒊍', '𒀝', '𒀳', '𒎎', '𒈲', '𒇵', '𒄢', '𒁽', '𒌅', '𒋺', '𒌦', '𒔸', '𒍒', '𒁶', '𒈥', '𒇺', '𒄧', '𒈭', '𒆳', '𒋠', '𒃹', '𒄐', '𒍑', '𒊢', '𒍝', '𒊠', '𒂯', '𒋰', '𒅜', '𒋭', '𒄴', '𒐢', '𒅏', '𒉺', '𒆰', '𒋧', '𒅖', '𒑘', '𒁔', '𒍪', '𒀯', '𒑖', '𒄸', '𒇯', '𒀏', '𒊾', '𒍞', '𒇇', '𒊿', '𒇳', '𒆏', '𒇋', '𒂅', '𒃷', '𒅾', '𒇼', '𒃋', '𒉚', '𒄘', '𒀜', '𒊽', '𒅓', '𒄨', '𒂟', '𒆍', '𒅗', '𒃽', '𒐊

In [135]:
from collections import defaultdict

def get_glyph_lens(dataset):
    counts = defaultdict(int)

    for split in dataset.keys():
        for example in dataset[split]:
            text = example["glyphs"]
            for token_ in ALL_SPECIAL_TOKS:
                text = text.replace(token_, "")
            if len(text) <= 64:
                counts["0-64"] += 1
            elif len(text) <= 128:
                counts["64-128"] += 1
            elif len(text) <= 256:
                counts["128-256"] += 1
            else:
                counts["mt256"] += 1
    return counts

get_glyph_lens(dataset)

defaultdict(int,
            {'0-64': 70212, '64-128': 20260, 'mt256': 4240, '128-256': 7412})

In [9]:
#len(pre_tokenizers.ByteLevel.alphabet()) + unique_glyph_count + len(list(ALL_SPECIAL_TOKS))
unique_glyph_count + len(list(ALL_SPECIAL_TOKS))

632

In [109]:
ALL_SPECIAL_TOKS = [
    '<s>',
    '<pad>',
    '</s>',
    '<unk>',
    '<mask>',
    "\n",
    "<SURFACE>",
    "<COLUMN>",
    "<BLANK_SPACE>",
    "<RULING>",
    "..."
]                        

def train_tokenizer():
    #vocab_size = len(pre_tokenizers.ByteLevel.alphabet()) + unique_glyph_count + len(list(SPECIAL_TOKENS)) -> 888
    trainer = trainers.BpeTrainer(
        initial_alphabet=list(unique_glyphs),
        #min_frequency=100,
        vocab_size=unique_glyph_count + len(list(ALL_SPECIAL_TOKS)),
        special_tokens=list(ALL_SPECIAL_TOKS),
        show_progress=True,
        #max_token_length=2,
    )
    #tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
    #tokenizer.normalizer = None
    #tokenizer.pre_tokenizer = ByteLevel()

    tokenizer = Tokenizer(models.BPE())
    tokenizer.add_special_tokens(list(ALL_SPECIAL_TOKS))
    #print(tokenizer.token_to_id('<s>'))
    #tokenizer.normalizer = normalizers.NFD()
    tokenizer.pre_tokenizer = pre_tokenizers.UnicodeScripts()
    tokenizer.post_processor = processors.RobertaProcessing(
        ("</s>", tokenizer.token_to_id('</s>')),
        ("<s>", tokenizer.token_to_id('<s>'))
    )
    tokenizer.decoder = decoders.BPEDecoder()

    tokenizer.train_from_iterator(iterator=glyph_texts, trainer=trainer)
    tokenizer.save("tokenizer.json")
    return tokenizer

tokenizer = train_tokenizer()

In [110]:
print(tokenizer.get_vocab_size())
glyph_vocab_items = [k for k in tokenizer.get_vocab().keys() if len(k) == 1]
print([x for x in glyph_vocab_items if x not in unique_glyphs])
print(len(glyph_vocab_items))
other_vocab_items = [k for k in tokenizer.get_vocab().keys() if len(k) != 1]
print(len(other_vocab_items))
other_vocab_items

632
['\n']
622
10


['<mask>',
 '<unk>',
 '<s>',
 '<pad>',
 '<BLANK_SPACE>',
 '</s>',
 '<COLUMN>',
 '<SURFACE>',
 '<RULING>',
 '...']

In [32]:
old_tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
old_tokenizer

XLMRobertaTokenizerFast(name_or_path='FacebookAI/xlm-roberta-base', vocab_size=250002, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	250001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [113]:
tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json")
tokenizer.add_special_tokens({'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['<SURFACE>', '<COLUMN>', '<BLANK_SPACE>', '<RULING>', '\n', '...']})
tokenizer.push_to_hub("SumerianGlyphTokenizer_Roberta")
#tokenizer.save("encoder_tokenizer.json")

CommitInfo(commit_url='https://huggingface.co/colesimmons/SumerianGlyphTokenizer_Roberta/commit/75fadd99143f2d2fc690c285b7228d262c524be1', commit_message='Upload tokenizer', commit_description='', oid='75fadd99143f2d2fc690c285b7228d262c524be1', pr_url=None, pr_revision=None, pr_num=None)

In [114]:
tokenizer

PreTrainedTokenizerFast(name_or_path='', vocab_size=632, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['<SURFACE>', '<COLUMN>', '<BLANK_SPACE>', '<RULING>', '\n', '...']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=False, single_word=False, normalized=False, spec

In [115]:
example = dataset["train"][0]["glyphs"]
without_special = example
for special_ in ALL_SPECIAL_TOKS:
    without_special = without_special.replace(special_, "")
print(example)
print(len(without_special))
encoded = tokenizer(example)
print(len(encoded["input_ids"]))
print(encoded)
print(tokenizer.decode(encoded["input_ids"]))
print(tokenizer.get_special_tokens_mask(encoded["input_ids"], already_has_special_tokens=True))

<SURFACE>
𒁹𒃢𒉺𒋼𒋛𒅗𒉌𒇻𒆠
𒁹𒃢𒆪𒁕𒈝𒉡𒌉
𒁹𒃢𒁹𒈧
𒂗𒀭𒈹
𒁹𒃢𒅗𒂵𒈬
𒁹𒄞𒊺
<SURFACE>
𒐊𒇻𒁹𒃢
𒀊𒁀𒆗𒆷𒈾𒃰
𒈬𒁺
𒌗𒂡𒀭𒎏𒀀𒍪
𒈬𒂗𒀭𒋀𒆠𒈦𒂊𒉌𒅆𒊒
<SURFACE>
𒌓𒁹𒄰
62
81
{'input_ids': [0, 6, 5, 87, 144, 389, 464, 442, 232, 366, 319, 267, 5, 87, 144, 274, 64, 333, 376, 475, 5, 87, 144, 87, 340, 5, 108, 35, 353, 5, 87, 144, 232, 126, 345, 5, 87, 187, 422, 5, 6, 5, 562, 319, 87, 144, 5, 16, 48, 260, 283, 356, 150, 5, 345, 88, 5, 485, 114, 35, 546, 12, 536, 5, 345, 108, 35, 427, 267, 339, 103, 366, 219, 402, 5, 6, 5, 482, 87, 204, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [61]:
print(dataset["train"][0]["glyphs"])
tokenizer(dataset["train"][0]["glyphs"])

<SURFACE>
𒁹𒃢𒉺𒋼𒋛𒅗𒉌𒇻𒆠
𒁹𒃢𒆪𒁕𒈝𒉡𒌉
𒁹𒃢𒁹𒈧
𒂗𒀭𒈹
𒁹𒃢𒅗𒂵𒈬
𒁹𒄞𒊺
<SURFACE>
𒐊𒇻𒁹𒃢
𒀊𒁀𒆗𒆷𒈾𒃰
𒈬𒁺
𒌗𒂡𒀭𒎏𒀀𒍪
𒈬𒂗𒀭𒋀𒆠𒈦𒂊𒉌𒅆𒊒
<SURFACE>
𒌓𒁹𒄰


{'input_ids': [0, 6, 5, 87, 144, 389, 464, 442, 232, 366, 319, 267, 5, 87, 144, 274, 64, 333, 376, 475, 5, 87, 144, 87, 340, 5, 108, 35, 353, 5, 87, 144, 232, 126, 345, 5, 87, 187, 422, 5, 6, 5, 562, 319, 87, 144, 5, 16, 48, 260, 283, 356, 150, 5, 345, 88, 5, 485, 114, 35, 546, 12, 536, 5, 345, 108, 35, 427, 267, 339, 103, 366, 219, 402, 5, 6, 5, 482, 87, 204, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [23]:
for id in encoded:
    print(tokenizer.decode(id))
    print("----")
decoded = tokenizer.decode(encoded, skip_special_tokens=True)
print(len((decoded.replace(" ", ""))))
decoded
#tokenizer.decode(encoded.ids, skip_special_tokens=False)

TypeError: argument 'ids': Can't extract `str` to `Vec`

In [30]:
ALL_SPECIAL_TOK_SET

{'\n',
 '...',
 '</s>',
 '<BLANK_SPACE>',
 '<COLUMN>',
 '<RULING>',
 '<SURFACE>',
 '<mask>',
 '<pad>',
 '<s>',
 '<unk>'}

In [ ]:
def get_special_token_ids(tokenizer, tokens):
    token_ids = []
    for token in tokens:
        token_id = tokenizer.token_to_id(token)
        if token_id is not None:
            token_ids.append((token, token_id))
        else:
            token_ids.append((token, 'Token not found in vocabulary'))
    return token_ids

#special_token_ids = get_special_token_ids(tokenizer, SPECIAL_TOKENS)
#special_token_ids = set([id for _, id in special_token_ids])
#special_token_ids



glyph_texts[0]